Libs para acesso ao AWS RDS 

In [386]:
import pymysql as pms
import pandas as pd

Informações para conexão

In [400]:
host_name = 'dhauz-instance.cutloqirhpd7.us-east-1.rds.amazonaws.com'
port      = '3306'
username  = 'candidate_user'
password  = 'D3@bGh664%$1VHv*'
db =        'db_hiring_test'

Conexão com Banco e dataset

In [401]:
cursor = pms.connect(host=host_name, user=username, passwd=password, database= db).cursor()

### ***Fase 1 – Análise sobre as transações:***


**Questão 1** -  Qual é o endereço (carteira) com maior volume de transações enviadas?

Em um primeiro momento, realizei essa query para verificar a maior quantidade de transações realizadas por carteira em toda a tabela 

In [314]:
query1 ='''
with aux as (
select AddressOrigin, Count(IdTransaction) as Num_Transaction from raw_transactions_table where Status = 'Confirmed' group by AddressOrigin order by Num_Transaction desc)

select Max(Num_Transaction) from aux
'''

In [316]:
cursor.execute(query1)
result_1= cursor.fetchone()

In [317]:
print(result_1)

(89,)


Depois, realizei a seleção da carteira com maior numero de transações por ordem decrescente de transações onde o status da transação fosse 'Confirmed'

In [312]:
query12 ='''
select AddressOrigin, Count(AddressOrigin) as Num_Transaction from raw_transactions_table where Status = 'Confirmed' group by AddressOrigin order by Num_Transaction desc
'''

In [313]:
cursor.execute(query12)
result_12= cursor.fetchone()
print(result_12)

('A-99', 89)


In [357]:
cursor.execute('''select AddressOrigin, Count(AddressOrigin) as Num_Transaction from raw_transactions_table where Status = 'Confirmed' group by AddressOrigin order by Num_Transaction desc''')
pd.DataFrame(cursor.fetchall(), columns = ['Carteira de Origem','Num Transação'])

,Carteira de Origem,Num Transação
0,A-99,89
1,A-72,85
2,A-83,85
3,A-25,81
4,A-78,81
...,...,...
96,A-100,41
97,A-26,36
98,A-35,35
99,A-30,32


A query acima demonstra que muitas carteiras possuem o mesmo numero de transações. 

**Questão 2** -  Qual é o dia do mês com maior volume de transações realizadas? 

In [90]:
query2 = '''
SELECT COUNT(EXTRACT(DAY FROM SentDate)) as Conta_dias, EXTRACT(DAY FROM SentDate) as Dia from raw_transactions_table
group by Dia Order By Conta_dias Desc
'''

In [112]:
cursor.execute(query2)
result_2= cursor.fetchall()
pd.DataFrame(result_2,columns = ['Conta Dias', 'Dias'])

,Conta Dias,Dias
0,288,7
1,284,2
2,274,15
3,262,27
4,262,12
5,239,1
6,234,20
7,232,14
8,228,13
9,220,19


Considerando apenas os dias do ano todo, o dia que possui maior numero de transações é o dia 7. 

Se passarmos a considerar o dia do mes, teremos a seguinte consideração: 

In [94]:
query21 = '''
SELECT COUNT(EXTRACT(DAY FROM SentDate)) as Conta_dias, EXTRACT(DAY FROM SentDate) as Dia, EXTRACT(MONTH FROM SentDate) as Mes, EXTRACT(Year FROM SentDate) as ano  from raw_transactions_table
group by Dia, Mes, ano Order By Conta_dias Desc
'''

In [111]:
cursor.execute(query21)
result_21= cursor.fetchall()
pd.DataFrame(result_21,columns = ['Conta Dias', 'Dias', 'Mes', 'Ano'])

,Conta Dias,Dias,Mes,Ano
0,288,7,1,2021
1,274,15,1,2021
2,262,27,1,2021
3,262,12,1,2021
4,240,2,1,2021
5,234,20,1,2021
6,232,14,1,2021
7,228,13,1,2021
8,220,19,1,2021
9,220,31,1,2021


Portanto, no dia 7 de janeiro do ano de 2021, foi o dia do ano com o maior numero de transações.

**Questão 3** - Em qual dia da semana geralmente mais transações são realizadas? 

In [108]:
query3 = '''
SELECT  DATE_FORMAT(SentDate, "%W") as dia_semana, count(*) as contagem from raw_transactions_table
group by dia_semana order by contagem desc
'''

In [110]:
cursor.execute(query3)
result_3= cursor.fetchall()
pd.DataFrame(result_3,columns = ['Dia da Semana', 'Contagem'])

,Dia da Semana,Contagem
0,Saturday,978
1,Wednesday,932
2,Friday,930
3,Thursday,911
4,Sunday,874
5,Tuesday,850
6,Monday,701


Portanto, geralmente são aos sabados os dias da semana em que mais são realizadas transações

**Questão 4** - Quais transações possuem condições atípicas e precisam ser validadas com o time
responsável pela disponibilização dos dados? 

Considerando o único campo que nos informa confirmação de transações, tem-se:

In [106]:
query4 = '''
SELECT * from raw_transactions_table where Status not in ('Confirmed', 'Denied')
'''

cursor.execute(query4)
result_4= cursor.fetchall()
pd.DataFrame(result_4, columns=['IdTransaction', 'AddressOrigin', 'AdressDestination', 'TotalSent', 'Status', 'SentDate', 'ImportDate'])

,IdTransaction,AddressOrigin,AdressDestination,TotalSent,Status,SentDate,ImportDate
0,ID1216,A-88,A-67,"502,360.00",Pending,2021-01-24 21:45:26,2021-01-31 23:59:59
1,ID2832,A-7,A-16,"119,209.00",Pending,2021-01-28 21:45:26,2021-01-31 23:59:59
2,ID2523,A-94,A-35,"773,785.00",Pending,2021-01-23 19:33:41,2021-01-31 23:59:59
3,ID2249,A-58,A-33,"940,414.00",Pending,2021-01-10 10:39:40,2021-01-31 23:59:59
4,ID746,A-98,A-89,"625,380.00",Pending,2021-01-10 18:57:38,2021-01-31 23:59:59
5,ID2985,A-31,A-69,"811,353.00",Pending,2021-01-01 20:08:01,2021-01-31 23:59:59
6,ID2873,A-2,A-1,"185,672.00",Pending,2021-01-28 02:57:50,2021-01-31 23:59:59
7,ID500,A-19,A-45,"966,464.00",Pending,2021-01-02 21:07:28,2021-01-31 23:59:59
8,ID1591,A-99,A-15,"881,559.00",Pending,2021-01-20 06:49:25,2021-01-31 23:59:59
9,ID2956,A-89,A-45,"355,195.00",Pending,2021-01-11 21:45:26,2021-01-31 23:59:59


Portanto, considerando que as transações que já foram negadas não serão mais avaliadas, as transações com status pendentes são as que precisam de observação do time responsável. 

**Questão 5** - Qual a carteira com o maior saldo final? (considere que todas as carteiras estejam zeradas no
início das análises e que seja possível existir saldo negativo) 

Para considerar o saldo da carteira, todos os valores devem ser somados, tanto os de entrada (positivos) quanto os de saída (negativos). Ainda, o saldo só se concretiza quando as transações são confirmadas. Portanto, deve-se contabilizar a saída de dinheiro da carteira e a entrada de dinheiro na carteira: 

In [410]:
query5 = '''
with balanco as (
select *, 
        TotalSent as recebido, 
        TotalSent as enviado
from raw_transactions_table
where Status = 'Confirmed'
), 

recebido_por_carteira as (
select sum(recebido) as sum_recebido, addressdestination from balanco group by addressdestination), 

enviado_por_carteira as (
select sum(enviado) as sum_enviado, addressOrigin from balanco group by addressOrigin),

correlaciona  as (
select  a.addressdestination as carteira, a.sum_recebido, b.sum_enviado
from recebido_por_carteira as a
INNER JOIN enviado_por_carteira as b 
on a.addressdestination = b.addressOrigin
)

select CARTEIRA, (sum_recebido - sum_enviado) as balanco_final from correlaciona order by balanco_final desc
'''
cursor.execute(query5)
result_5= cursor.fetchall()
pd.DataFrame(result_5,columns = ['carteira', 'Balanço'])

,carteira,Balanço
0,A-30,25188.0
1,A-67,22811.0
2,A-12,22502.0
3,A-35,20608.0
4,A-73,20117.0
...,...,...
96,A-83,-17925.0
97,A-19,-19404.0
98,A-9,-19901.0
99,A-99,-20366.0


### ***Fase 2 – Análise sobre as taxas:***

**Questão 1** - Considerando que a carteira origem é responsável por pagar as taxas de envio, qual carteira
seria responsável pelo maior pagamento de taxas em janeiro de 2021? 

Apesar de ser mais simples colocar direto no CASE ... WHEN o valor das taxas, criei uma relação entre as duas tabelas. 

In [350]:
query6 = '''
With Cria_range as (
select *,
    CASE
       WHEN  replace(TotalSent,',','') >= 0     and replace(TotalSent,',','') <= 160000    then '1'
        WHEN replace(TotalSent,',','') > 160000 and replace(TotalSent,',','') <= 340000    then '2'
        WHEN replace(TotalSent,',','') > 340000 and replace(TotalSent,',','') <= 500000    then '3'
        WHEN replace(TotalSent,',','') > 500000 and replace(TotalSent,',','') <= 670000    then '4'
        WHEN replace(TotalSent,',','') > 670000 and replace(TotalSent,',','') <= 833000    then '5'
        WHEN replace(TotalSent,',','') > 833000 and replace(TotalSent,',','') <= 99000000  then '6'
        Else '0' 
    END AS INDEX_FEE
from raw_transactions_table
), 

range_fee as (
Select *,
    CASE
        WHEN `range-start` >= 0     and `range-end` <= 160000    then '1'
        WHEN `range-start` > 160000 and `range-end` <= 340000    then '2'
        WHEN `range-start` > 340000 and `range-end` <= 500000    then '3'
        WHEN `range-start` > 500000 and `range-end` <= 670000    then '4'
        WHEN `range-start` > 670000 and `range-end` <= 833000    then '5'
        WHEN `range-start` > 833000 and `range-end` <= 99000000  then '6'
        Else '0' 
    END AS INDEX_FEE
from raw_transactions_fee
), 

join_fee as (
select a.*, b.`fee-percentage` as fee
from Cria_range as a
left join range_fee as b
on a.INDEX_FEE = b.INDEX_FEE
)

SELECT AddressOrigin, sum(replace(TotalSent,',','')*fee/100) as Taxas, DATE_FORMAT(SentDate, '%m%Y') as data FROM join_fee where DATE_FORMAT(SentDate, '%m%Y') = '012021' 
group by AddressOrigin order by Taxas Desc ;
'''

cursor.execute(query6)
result_6= cursor.fetchall()
pd.DataFrame(result_6, columns=['addressOrigin', 'Taxas', 'data'])


,addressOrigin,Taxas,data
0,A-99,1993046.36,012021
1,A-83,1765339.54,012021
2,A-25,1713452.80,012021
3,A-72,1675731.68,012021
4,A-68,1668694.14,012021
...,...,...,...
95,A-100,894069.84,012021
96,A-26,834148.80,012021
97,A-67,730342.14,012021
98,A-35,692711.98,012021


**Questão 2** - Considerando que a carteira origem é responsável por pagar as taxas de envio, qual carteira
seria responsável pelo maior pagamento de taxas em fevereiro de 2021? 

In [351]:
query6 = '''
With Cria_range as (
select *,
    CASE
        WHEN replace(TotalSent,',','') >= 0     and replace(TotalSent,',','') <= 160000    then '1'
        WHEN replace(TotalSent,',','') > 160000 and replace(TotalSent,',','') <= 340000    then '2'
        WHEN replace(TotalSent,',','') > 340000 and replace(TotalSent,',','') <= 500000    then '3'
        WHEN replace(TotalSent,',','') > 500000 and replace(TotalSent,',','') <= 670000    then '4'
        WHEN replace(TotalSent,',','') > 670000 and replace(TotalSent,',','') <= 833000    then '5'
        WHEN replace(TotalSent,',','') > 833000 and replace(TotalSent,',','') <= 99000000  then '6'
        Else '0' 
    END AS INDEX_FEE
from raw_transactions_table
), 

range_fee as (
Select *,
    CASE
        WHEN `range-start` >= 0     and `range-end` <= 160000    then '1'
        WHEN `range-start` > 160000 and `range-end` <= 340000    then '2'
        WHEN `range-start` > 340000 and `range-end` <= 500000    then '3'
        WHEN `range-start` > 500000 and `range-end` <= 670000    then '4'
        WHEN `range-start` > 670000 and `range-end` <= 833000    then '5'
        WHEN `range-start` > 833000 and `range-end` <= 99000000  then '6'
        Else '0' 
    END AS INDEX_FEE
from raw_transactions_fee
), 

join_fee as (
select a.*, b.`fee-percentage` as fee
from Cria_range as a
left join range_fee as b
on a.INDEX_FEE = b.INDEX_FEE
)

SELECT AddressOrigin, sum(replace(TotalSent,',','')*fee/100) as Taxas, DATE_FORMAT(SentDate, '%m%Y') as data FROM join_fee where DATE_FORMAT(SentDate, '%m%Y') = '022021' 
group by AddressOrigin order by Taxas Desc ;
'''

cursor.execute(query6)
result_6= cursor.fetchall()
pd.DataFrame(result_6, columns=['addressOrigin', 'Taxas', 'data'])

,addressOrigin,Taxas,data
0,A-29,107830.31,022021
1,A-30,106344.95,022021
2,A-68,104406.01,022021
3,A-27,102267.66,022021
4,A-57,92955.41,022021
...,...,...,...
76,A-93,10023.10,022021
77,A-84,8164.70,022021
78,A-56,4219.70,022021
79,A-25,2279.10,022021


**Questão 3** - Qual é o id da transação com a maior taxa paga?? 

In [353]:
query6 = '''
With Cria_range as (
select *,
    CASE
        WHEN replace(TotalSent,',','') >= 0     and replace(TotalSent,',','') <= 160000    then '1'
        WHEN replace(TotalSent,',','') > 160000 and replace(TotalSent,',','') <= 340000    then '2'
        WHEN replace(TotalSent,',','') > 340000 and replace(TotalSent,',','') <= 500000    then '3'
        WHEN replace(TotalSent,',','') > 500000 and replace(TotalSent,',','') <= 670000    then '4'
        WHEN replace(TotalSent,',','') > 670000 and replace(TotalSent,',','') <= 833000    then '5'
        WHEN replace(TotalSent,',','') > 833000 and replace(TotalSent,',','') <= 99000000  then '6'
        Else '0' 
    END AS INDEX_FEE
from raw_transactions_table
), 

range_fee as (
Select *,
    CASE
        WHEN `range-start` >= 0     and `range-end` <= 160000    then '1'
        WHEN `range-start` > 160000 and `range-end` <= 340000    then '2'
        WHEN `range-start` > 340000 and `range-end` <= 500000    then '3'
        WHEN `range-start` > 500000 and `range-end` <= 670000    then '4'
        WHEN `range-start` > 670000 and `range-end` <= 833000    then '5'
        WHEN `range-start` > 833000 and `range-end` <= 99000000  then '6'
        Else '0' 
    END AS INDEX_FEE
from raw_transactions_fee
), 

join_fee as (
select a.*, b.`fee-percentage` as fee
from Cria_range as a
left join range_fee as b
on a.INDEX_FEE = b.INDEX_FEE
)

SELECT IdTransaction, Max(replace(TotalSent,',','')*fee/100) as Max_Taxa FROM join_fee 
group by IdTransaction order by Max_Taxa Desc limit 1 ;
'''

cursor.execute(query6)
result_6= cursor.fetchall()
pd.DataFrame(result_6, columns=['IdTransaction', 'Max_Taxa'])

,IdTransaction,Max_Taxa
0,ID635,33485.05


**Questão 4** - Qual é a média de taxa paga considerando todas as transações realizadas??? 

In [355]:
query6 = '''
With Cria_range as (
select *,
    CASE
        WHEN replace(TotalSent,',','') >= 0     and replace(TotalSent,',','') <= 160000    then '1'
        WHEN replace(TotalSent,',','') > 160000 and replace(TotalSent,',','') <= 340000    then '2'
        WHEN replace(TotalSent,',','') > 340000 and replace(TotalSent,',','') <= 500000    then '3'
        WHEN replace(TotalSent,',','') > 500000 and replace(TotalSent,',','') <= 670000    then '4'
        WHEN replace(TotalSent,',','') > 670000 and replace(TotalSent,',','') <= 833000    then '5'
        WHEN replace(TotalSent,',','') > 833000 and replace(TotalSent,',','') <= 99000000  then '6'
        Else '0' 
    END AS INDEX_FEE
from raw_transactions_table
), 

range_fee as (
Select *,
    CASE
        WHEN `range-start` >= 0     and `range-end` <= 160000    then '1'
        WHEN `range-start` > 160000 and `range-end` <= 340000    then '2'
        WHEN `range-start` > 340000 and `range-end` <= 500000    then '3'
        WHEN `range-start` > 500000 and `range-end` <= 670000    then '4'
        WHEN `range-start` > 670000 and `range-end` <= 833000    then '5'
        WHEN `range-start` > 833000 and `range-end` <= 99000000  then '6'
        Else '0' 
    END AS INDEX_FEE
from raw_transactions_fee
), 

join_fee as (
select a.*, b.`fee-percentage` as fee
from Cria_range as a
left join range_fee as b
on a.INDEX_FEE = b.INDEX_FEE
)

SELECT sum(replace(TotalSent,',','')*fee/100)/count(*) as mean_Taxa FROM join_fee 
'''

cursor.execute(query6)
result_6= cursor.fetchall()
pd.DataFrame(result_6, columns=['mean_Taxa'])

,mean_Taxa
0,21713.317191


### ***Fase 3 – Arquitetura:***

De maneira simplificada, a criação de Cloud Functions (ambiente GCP) é o suficiente para suprir essa demanda. A mesma pode abrigar diversos códigos que buscam na AWS as informações necessárias. Ainda, a Cloud Function com o código python pode ser capaz de escrever esses dados no BigQuery, criando uma tabela ou anexando os novos dados a uma tabela já existente. 

Caso seja necessário o carregamento desses dados em streming, pode-se aliar o Pub/Sub à Cloud Function para que atue como Trigger, disparando a execução do processo assim que algum estimulo chegasse, como atualização de dados na origem AWS. 

Ainda, uma terceira possibilidade é a criação de uma DAG para a leitura, tratamento e escrita desses dados, sendo executada pelo AirFlow. 